## Evaluating Random Integral

### The given integral:

\begin{equation}
K_n = \displaystyle\int_a^b \frac{x^n}{z + ax} dx 
\end{equation} 

### The recursion relation:

\begin{equation}
K_{n+1} = \frac{1}{\alpha} \frac{b^{n+1} - a^{n+1}}{n+1} - \frac{z}{\alpha}K_n 
\end{equation} 

In [1]:
import numpy as np
import matplotlib.pyplot as plt

In [52]:
alpha = 2.0
z = 1.0

In [53]:
def func(n:int, x:float):
    return pow(x, n)/(z + alpha * x)

In [54]:
def upward_recursion(order:int, u_lim:int = 1, l_lim:int = 0):
    k = np.zeros(order+1)
    try:
        k_not = 1/alpha * np.log((z+alpha*u_lim)/(z+alpha*l_lim))
        k[0] = k_not
        if order <= 0 :
            return k_not
        
        for n in range(1, order + 1):
            k_next = 1/alpha * (pow(u_lim, n) - pow(l_lim, n))/n - (z/alpha) * k_not
            k[n] = k_next
            k_not = k_next

        return k

    except ZeroDivisionError:
        print("Enter non zero value for alpha and z")
        return 0.0


In [55]:
def standard_eval(order:int, steps:int = 10000, u_lim:int = 1, l_lim:int = 0):
    k_std = np.zeros(order+1)
    dx = (u_lim - l_lim)/steps
    for n in range (order+1):
        integral = 0.0
        for step in range(steps):
            x = (0.5 + step)*dx
            integral += func(n, x)
        k_std[n] = integral*dx
    return k_std

## Evaluate series:

\begin{equation}
Sum = K_{1} + K_2 + K_3 + ... + K_n
\end{equation}

In [56]:
def evaluate_series(terms:int):
    sum_up = sum(np.array(upward_recursion(terms)))
    sum_std = sum(np.array(standard_eval(terms)))
    return sum_up, sum_std

In [57]:
result_up, result_std = evaluate_series(10)
print(f'Error: {result_std - result_up}')

Error: -7.037037619284092e-09


Testing upward recursion for

\begin{equation} {\frac{\alpha}{z}} < 1 \end{equation}

In [58]:
alpha = 1e-4
result_up, result_std = evaluate_series(10)
print(f'Error: {result_std - result_up}')

Error: 1.1013182817544156e+27


In [59]:
def downward_recursion(order:int, u_lim:int = 1, l_lim:int = 0):
    epsilon = 1e-16
    sum_start = 0
    sum_end = int((np.log(epsilon))/(np.log(alpha/z)))
    k = np.zeros(order + 1)
    k_next = 0.0
    for m in range(sum_end + 1):
        k_next += (pow(-alpha, m)/pow(z, m+1)) * (pow(u_lim, order+m+2) - pow(l_lim, order+m+2))/(order+m+2)

    for n in range(order, -1, -1):
        k_prev =  1/z * (pow(u_lim, n+1) - pow(l_lim, n+1))/(n+1) - alpha/z * k_next
        k[n] = k_prev
        k_next = k_prev
    return k

In [60]:
alpha = 1e-4
sum_down = sum(np.array(downward_recursion(10)))
sum_std = sum(np.array(standard_eval(10)))
print(f'Downward Recursion error: {sum_std - sum_down}')

Downward Recursion error: -2.2497284923872485e-08
